In [1]:
import os 
import torch 
from random import randint
from datasets import load_dataset
from tqdm.auto import tqdm 

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
# "/home/haiqv/llm_finetuning_book/03_5_Llama/llama-3.2-korean-3b-hf-20-epoch/checkpoint-4740"
# model_name = "/home/haiqv/llm_finetuning_book/03_5_Llama/llama-3.2-korean-3b-hf-20-epoch/checkpoint-4740"
model_name = "./llama-3.2-korean-3b-hf-20-epoch/checkpoint-4740"
if not os.path.exists(model_name):
    print(f"Path does not exist: {model_name}")
else:
    print(f"Path exists: {model_name}")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    use_cache=False,
    device_map="auto"
)

test_dataset = load_dataset(
    "json",
    data_files=os.path.join("", "./test_dataset.json"),
    split="train",
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

test_dataset = load_dataset("json", 
                            split="train",
                            data_files="test_dataset.json")

random_index = randint(0, len(test_dataset))
messages = test_dataset[random_index]["messages"][:2]

terminators = [
    tokenizer.eos_token_id,
]

input_ids = tokenizer.apply_chat_template(messages,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)
response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_index]['messages'][1]['content']}")
print(f"정답:\n{test_dataset[random_index]['messages'][2]['content']}")
print(f"생성:\n{tokenizer.decode(response,skip_special_tokens=True)}")

/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path exists: ./llama-3.2-korean-3b-hf-20-epoch/checkpoint-4740


Loading checkpoint shards: 100%|██████████| 3/3 [00:26<00:00,  8.80s/it]
Generating train split: 2116 examples [00:00, 184135.51 examples/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


질문:
와사비와 겨자의 차이점이 무엇인가요?
정답:
와사비는 고추냉이를 갈아서 사용하는 반면, 겨자는 겨자의 씨앗을 갈아서 만듭니다. 와사비는 찬물에 개어서 신랄한 향을 내며, 겨자는 뜨거운 물에 발효시켜 강한 매운 맛을 냅니다. 따라서, 와사비는 비린내를 없애고 가벼운 맛을 내는 간장소스와 같은 요리에 사용되며, 겨자는 냉채나 무침, 돈까스 소스 등 강한 맛을 내는 요리에 사용됩니다. 또한, 와사비는 향이 강한 반면 겨자는 맛이 강하다고 할 수 있습니다.
생성:
1. 와사비는 일본에서 자란 식물으로, 겨자와 비슷한 모양을 가지고 있습니다. 그러나 사용 용도가 다르며, 주로 일본 요리에서 사용됩니다. 
2. 겨자는 인도에서 자라는 식물로, 매우 매운 맛을 가지고 있습니다. 중국 요리에서 사용되는 경우도 많습니다. 
3. 사프란은 프랑스에서 자라는 식물로,과일과 채소를 잘라내어 염산에 담가 매운 맛을 내는 음식입니다. 주로 프랑스 요리에서 사용됩니다. 
4. 고추는 중국에서 자라는 식물로, 매운 맛이 강합니다. 한국, 일본, 미국, 중국 등에서 다양한 요리로 즐겨먹고 있습니다. 

위의 식물들은 모두 매운 맛을 가지고 있지만, 사용 용도와 조리 방법에 차이가 있습니다.
